In [1]:
from pyspark.sql.functions import isnull,sum

In [2]:
# Importing pyspark and starting session to use sprak functionality 
import pyspark 
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,when,isnan,count
from pyspark.sql.types import IntegerType, StructType,StructField, FloatType , DoubleType
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler,StandardScaler
from pyspark.ml.stat import Correlation
from pyspark.sql.functions import regexp_replace
from pyspark.ml.regression import LinearRegression 
spark = SparkSession.builder.appName("football").getOrCreate()


In [39]:
# reading csv file
dataset = spark.read.csv('Data.csv',inferSchema=True,header=True) # infer schema automatocally guess column datatypes
dataset.show(5)

+-------+-------+-----+-----------------+--------------+-------------+----+-----+-----+----------------+-----+--------+-------------------+-----------------------+----+
|Country| League| Club|     Player Names|Matches_Played|Substitution |Mins|Goals|   xG|xG Per Avg Match|Shots|OnTarget|Shots Per Avg Match|On Target Per Avg Match|Year|
+-------+-------+-----+-----------------+--------------+-------------+----+-----+-----+----------------+-----+--------+-------------------+-----------------------+----+
|  Spain|La Liga|(BET)|  Juanmi Callejon|            19|           16|1849|   11| 6.62|            0.34|   48|      20|               2.47|                   1.03|2016|
|  Spain|La Liga|(BAR)|Antoine Griezmann|            36|            0|3129|   16|11.86|            0.36|   88|      41|               2.67|                   1.24|2016|
|  Spain|La Liga|(ATL)|      Luis Suarez|            34|            1|2940|   28|23.21|            0.75|  120|      57|               3.88|                

In [40]:
dataset.printSchema()

root
 |-- Country: string (nullable = true)
 |-- League: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- Player Names: string (nullable = true)
 |-- Matches_Played: integer (nullable = true)
 |-- Substitution : integer (nullable = true)
 |-- Mins: integer (nullable = true)
 |-- Goals: integer (nullable = true)
 |-- xG: double (nullable = true)
 |-- xG Per Avg Match: double (nullable = true)
 |-- Shots: integer (nullable = true)
 |-- OnTarget: integer (nullable = true)
 |-- Shots Per Avg Match: double (nullable = true)
 |-- On Target Per Avg Match: double (nullable = true)
 |-- Year: integer (nullable = true)



In [41]:
dataset.count()

660

In [42]:
null_count = dataset.select([sum(isnull(c).cast("int")).alias(c) for c in dataset.columns])

# print the count of null values in each column
null_count.show()

+-------+------+----+------------+--------------+-------------+----+-----+---+----------------+-----+--------+-------------------+-----------------------+----+
|Country|League|Club|Player Names|Matches_Played|Substitution |Mins|Goals| xG|xG Per Avg Match|Shots|OnTarget|Shots Per Avg Match|On Target Per Avg Match|Year|
+-------+------+----+------------+--------------+-------------+----+-----+---+----------------+-----+--------+-------------------+-----------------------+----+
|      0|     0|   0|           0|             0|            0|   0|    0|  0|               0|    0|       0|                  0|                      0|   0|
+-------+------+----+------------+--------------+-------------+----+-----+---+----------------+-----+--------+-------------------+-----------------------+----+



In [43]:
df = dataset.drop("Substitution ", "Mins")
df.show()

+-------+-------+------+-----------------+--------------+-----+-----+----------------+-----+--------+-------------------+-----------------------+----+
|Country| League|  Club|     Player Names|Matches_Played|Goals|   xG|xG Per Avg Match|Shots|OnTarget|Shots Per Avg Match|On Target Per Avg Match|Year|
+-------+-------+------+-----------------+--------------+-----+-----+----------------+-----+--------+-------------------+-----------------------+----+
|  Spain|La Liga| (BET)|  Juanmi Callejon|            19|   11| 6.62|            0.34|   48|      20|               2.47|                   1.03|2016|
|  Spain|La Liga| (BAR)|Antoine Griezmann|            36|   16|11.86|            0.36|   88|      41|               2.67|                   1.24|2016|
|  Spain|La Liga| (ATL)|      Luis Suarez|            34|   28|23.21|            0.75|  120|      57|               3.88|                   1.84|2016|
|  Spain|La Liga| (CAR)|     Ruben Castro|            32|   13|14.06|            0.47|  117|  

In [44]:
df = df.withColumnRenamed("xG", "Expected goals").withColumnRenamed("xG Per Avg Match", "Expected Goals AVG Match")
df.show(5)

+-------+-------+-----+-----------------+--------------+-----+--------------+------------------------+-----+--------+-------------------+-----------------------+----+
|Country| League| Club|     Player Names|Matches_Played|Goals|Expected goals|Expected Goals AVG Match|Shots|OnTarget|Shots Per Avg Match|On Target Per Avg Match|Year|
+-------+-------+-----+-----------------+--------------+-----+--------------+------------------------+-----+--------+-------------------+-----------------------+----+
|  Spain|La Liga|(BET)|  Juanmi Callejon|            19|   11|          6.62|                    0.34|   48|      20|               2.47|                   1.03|2016|
|  Spain|La Liga|(BAR)|Antoine Griezmann|            36|   16|         11.86|                    0.36|   88|      41|               2.67|                   1.24|2016|
|  Spain|La Liga|(ATL)|      Luis Suarez|            34|   28|         23.21|                    0.75|  120|      57|               3.88|                   1.84|2016

In [45]:
df.printSchema()

root
 |-- Country: string (nullable = true)
 |-- League: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- Player Names: string (nullable = true)
 |-- Matches_Played: integer (nullable = true)
 |-- Goals: integer (nullable = true)
 |-- Expected goals: double (nullable = true)
 |-- Expected Goals AVG Match: double (nullable = true)
 |-- Shots: integer (nullable = true)
 |-- OnTarget: integer (nullable = true)
 |-- Shots Per Avg Match: double (nullable = true)
 |-- On Target Per Avg Match: double (nullable = true)
 |-- Year: integer (nullable = true)



In [46]:
column_names = df.columns

# print the column names
print(column_names)

['Country', 'League', 'Club', 'Player Names', 'Matches_Played', 'Goals', 'Expected goals', 'Expected Goals AVG Match', 'Shots', 'OnTarget', 'Shots Per Avg Match', 'On Target Per Avg Match', 'Year']


In [47]:
df.toPandas().to_excel('final.xlsx', sheet_name = 'Sheet1', index = False)

In [48]:
strinx = StringIndexer(inputCols = ["Country","League","Club","Player Names"],outputCols =["Country_trans","League_trans","Club_trans","Player Names_trans"] )

In [49]:
df = strinx.fit(df).transform(df)

In [50]:
df.show()

+-------+-------+------+-----------------+--------------+-----+--------------+------------------------+-----+--------+-------------------+-----------------------+----+-------------+------------+----------+------------------+
|Country| League|  Club|     Player Names|Matches_Played|Goals|Expected goals|Expected Goals AVG Match|Shots|OnTarget|Shots Per Avg Match|On Target Per Avg Match|Year|Country_trans|League_trans|Club_trans|Player Names_trans|
+-------+-------+------+-----------------+--------------+-----+--------------+------------------------+-----+--------+-------------------+-----------------------+----+-------------+------------+----------+------------------+
|  Spain|La Liga| (BET)|  Juanmi Callejon|            19|   11|          6.62|                    0.34|   48|      20|               2.47|                   1.03|2016|          3.0|         2.0|      50.0|             289.0|
|  Spain|La Liga| (BAR)|Antoine Griezmann|            36|   16|         11.86|                    0.

In [51]:
onenc = OneHotEncoder(inputCols = ["Country_trans","League_trans","Club_trans","Player Names_trans"],
                      outputCols =["Country_VEC","League_VEC","Club_VEC"," Player Names_VEC"] )
df = onenc.fit(df).transform(df)

In [52]:
df.show(5,vertical=True)

-RECORD 0-------------------------------------
 Country                  | Spain             
 League                   | La Liga           
 Club                     | (BET)             
 Player Names             | Juanmi Callejon   
 Matches_Played           | 19                
 Goals                    | 11                
 Expected goals           | 6.62              
 Expected Goals AVG Match | 0.34              
 Shots                    | 48                
 OnTarget                 | 20                
 Shots Per Avg Match      | 2.47              
 On Target Per Avg Match  | 1.03              
 Year                     | 2016              
 Country_trans            | 3.0               
 League_trans             | 2.0               
 Club_trans               | 50.0              
 Player Names_trans       | 289.0             
 Country_VEC              | (8,[3],[1.0])     
 League_VEC               | (27,[2],[1.0])    
 Club_VEC                 | (179,[50],[1.0])  
  Player Name

In [53]:
column_names = df.columns

# print the column names
print(column_names)

['Country', 'League', 'Club', 'Player Names', 'Matches_Played', 'Goals', 'Expected goals', 'Expected Goals AVG Match', 'Shots', 'OnTarget', 'Shots Per Avg Match', 'On Target Per Avg Match', 'Year', 'Country_trans', 'League_trans', 'Club_trans', 'Player Names_trans', 'Country_VEC', 'League_VEC', 'Club_VEC', ' Player Names_VEC']


In [54]:
df_dropped = df.drop("Country","League","Club","Player Names")
df_dropped.show()

+--------------+-----+--------------+------------------------+-----+--------+-------------------+-----------------------+----+-------------+------------+----------+------------------+-------------+--------------+-----------------+-----------------+
|Matches_Played|Goals|Expected goals|Expected Goals AVG Match|Shots|OnTarget|Shots Per Avg Match|On Target Per Avg Match|Year|Country_trans|League_trans|Club_trans|Player Names_trans|  Country_VEC|    League_VEC|         Club_VEC| Player Names_VEC|
+--------------+-----+--------------+------------------------+-----+--------+-------------------+-----------------------+----+-------------+------------+----------+------------------+-------------+--------------+-----------------+-----------------+
|            19|   11|          6.62|                    0.34|   48|      20|               2.47|                   1.03|2016|          3.0|         2.0|      50.0|             289.0|(8,[3],[1.0])|(27,[2],[1.0])| (179,[50],[1.0])|(443,[289],[1.0])|
|   

In [55]:
new_column_names = df_dropped.columns


In [56]:
Va = VectorAssembler()
vA = Va.setParams(inputCols=new_column_names, outputCol='features')
df_dropped = vA.transform(df_dropped)
df.show(5,vertical=True)

-RECORD 0-------------------------------------
 Country                  | Spain             
 League                   | La Liga           
 Club                     | (BET)             
 Player Names             | Juanmi Callejon   
 Matches_Played           | 19                
 Goals                    | 11                
 Expected goals           | 6.62              
 Expected Goals AVG Match | 0.34              
 Shots                    | 48                
 OnTarget                 | 20                
 Shots Per Avg Match      | 2.47              
 On Target Per Avg Match  | 1.03              
 Year                     | 2016              
 Country_trans            | 3.0               
 League_trans             | 2.0               
 Club_trans               | 50.0              
 Player Names_trans       | 289.0             
 Country_VEC              | (8,[3],[1.0])     
 League_VEC               | (27,[2],[1.0])    
 Club_VEC                 | (179,[50],[1.0])  
  Player Name

In [57]:
splt = df_dropped.randomSplit([0.7,0.3])
train_df= splt[0]
test_df = splt[1]

In [58]:
from pyspark.ml.regression import LinearRegression
regre = LinearRegression(featuresCol = 'features', labelCol='Goals', maxIter=10)
Regremodel = regre.fit(train_df)
print("Intercept: " + str(Regremodel.intercept))

Intercept: -144.06111275758917


In [59]:
summary = Regremodel.summary
summary.rootMeanSquaredError
summary.r2

0.9894033858602438

In [67]:
from pyspark.ml.regression import RandomForestRegressor
random_forest_regressor = RandomForestRegressor(featuresCol='features', labelCol='Goals', numTrees=100,  maxBins=444)
random_forest_regression_model = random_forest_regressor.fit(train_df)

In [68]:
from pyspark.ml.evaluation import RegressionEvaluator

# Make predictions on the test data
predictions = random_forest_regression_model.transform(test_df)

# Create a RegressionEvaluator with the appropriate metric (e.g., R2, RMSE, MSE, or MAE)
evaluator = RegressionEvaluator(labelCol="Goals", predictionCol="prediction", metricName="r2")

# Calculate the accuracy (here, we use RMSE as an example)
r2 = evaluator.evaluate(predictions)
print("R Square (R^2) on test data = %g" % r2)


R Square (R^2) on test data = 0.852802


In [69]:
from pyspark.ml.regression import DecisionTreeRegressor
DecisionTree_regressor = DecisionTreeRegressor(featuresCol='features', labelCol='Goals', maxBins=444)
DecisionTree_regression_model = DecisionTree_regressor.fit(train_df)

In [70]:
from pyspark.ml.evaluation import RegressionEvaluator

# Make predictions on the test data
predictions = DecisionTree_regression_model.transform(test_df)

# Create a RegressionEvaluator with the appropriate metric (e.g., R2, RMSE, MSE, or MAE)
evaluator = RegressionEvaluator(labelCol="Goals", predictionCol="prediction", metricName="r2")

# Calculate the accuracy (here, we use RMSE as an example)
r2 = evaluator.evaluate(predictions)
print("R Square (R^2) on test data = %g" % r2)


R Square (R^2) on test data = 0.996264
